# Day 3: Supply II - Enhancing the model

We will continue our modeling exercise by adding carbon taxes, renewable subsidies, and investment to the model.

This will allow us to consider the policy impacts of alternative energy transition policies.

The data and code are based on the paper "The Efficiency and Sectoral Distributional Implications of Large-Scale Renewable Policies," by Mar Reguant.

We first load relevant libraries, same as last session.

In [ ]:
#using Pkg
#Pkg.add(["DataFrames", "CSV", "JuMP", "Ipopt", "Plots", "Printf"])


In [1]:
using DataFrames
using CSV
using JuMP
using Ipopt
using Plots
using Printf

## Building the model

We load the same data as last week, and also clean it up to simplify it further and create the demand and import curves.

In [2]:
dfclust = CSV.read("data_jaere_clustered_500.csv", DataFrame);

# Re-scaling (we multiply by 8.76 to make it into a full year of hours (divided by 1000))
dfclust.weights = 8.76 * dfclust.weights / sum(dfclust.weights);

# Here only one demand type to make it easier
dfclust.demand = dfclust.q_residential + dfclust.q_commercial + dfclust.q_industrial;

# Calibrate demand based on elasticities (using 0.1 here as only one final demand)
elas = [.1, .2, .5, .3];
dfclust.b = elas[1] * dfclust.demand ./ dfclust.price;  # slope
dfclust.a = dfclust.demand + dfclust.b .* dfclust.price;  # intercept

# Calibrate imports (using elas 0.3)
dfclust.bm = elas[4] * dfclust.imports ./ dfclust.price;  # slope
dfclust.am = dfclust.imports - dfclust.bm .* dfclust.price;  # intercept

In [ ]:
dfclust

The technology file now includes the fixed cost of building new power plants (technologies 3-5). Note that we added an additional row for new natural gas plants.

We will use an annualization factor to pro-rate the importance of fixed costs for one year.

In [3]:
tech = CSV.read("data_technology.csv", DataFrame);
afactor = (1 - (1 / (1.05^20.0))) / 0.05;
tech.F = tech.F ./afactor;
tech.F2 = tech.F2 ./afactor;
tech



,techname,heatrate,heatrate2,F,capLB,capUB,new,renewable
,String15,Float64,Float64,Float64,Float64,Float64,Int64,Int64
1,Hydro/Nuclear,10.0,0.0,0.0,1.0,1.0,0,0
2,Existing 1,6.67199,0.0929123,0.0,11.5,11.5,0,0
3,Existing 2,9.79412,0.286247,0.0,14.5,14.5,0,0
4,Existing 3,13.8181,20.5352,0.0,0.578,0.578,0,0
5,New Gas,6.6,0.0,78.4773,0.0,100.0,1,0
6,Wind,0.0,0.0,100.303,0.0,100.0,1,1
7,Solar,0.0,0.0,100.303,0.0,100.0,1,1


In [ ]:
describe(tech)

### Adding investment

We are now ready to clear the market. We will **maximize welfare** using a non-linear solver.

$$ \max \ CS - Costs - Fixed Costs \\

\text{s.t.} \ \text{operational constraints, market clearing}. $$

Notice that we added the fixed costs to the problem, as we will be solving for the optimal level of wind and solar investment.

In [ ]:
## Clear market based on cost minimization
function clear_market_invest(data::DataFrame, tech::DataFrame)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

    
    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);

    
    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                (gross_surplus[t] - costs[t] - ) for t=1:T)
             - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw 
             - tech.F2[5]*gas_gw^2  - tech.F2[6]*wind_gw^2 - tech.F2[7]*solar_gw^2);

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * price[t]);
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
                (data.a[t] - demand[t]) * demand[t] / data.b[t] 
            + demand[t]^2/(2*data.b[t]));

    # Define cost
    @constraint(model, [t=1:T], costs[t] ==
                    sum(tech.c[i] * quantity[t,i]
                    + tech.c2[i] * quantity[t,i]^2/2 for i=1:I)
        + (imports[t] - data.am[t])^2/(2 * data.bm[t]))

    # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);


    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        cost = JuMP.value.(costs);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

In [ ]:
results_min = clear_market_invest(dfclust, tech)

## Environmental regulation

We will begin by adding carbon taxes to the model.



In [ ]:
function clear_market_invest_tax(data::DataFrame, tech::DataFrame,tax=30.0)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);

    #New Variable
    @variable(model, totale[1:T]); #total emissions
    

    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                (gross_surplus[t] - costs[t] - tax*totale[t]) for t=1:T)
             - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw 
             - tech.F2[5]*gas_gw^2  - tech.F2[6]*wind_gw^2 - tech.F2[7]*solar_gw^2);

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * price[t]);
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
                (data.a[t] - demand[t]) * demand[t] / data.b[t] 
            + demand[t]^2/(2*data.b[t]));

    # Define cost
    @constraint(model, [t=1:T], costs[t] ==
                    sum(tech.c[i] * quantity[t,i]
                    + tech.c2[i] * quantity[t,i]^2/2 for i=1:I)
        + (imports[t] - data.am[t])^2/(2 * data.bm[t]))

    # Define emissions 
    @constraint(model, [t=1:T], totale[t] ==
                    sum(tech.e[i] * quantity[t,i] for i=1:I))

        # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);


    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        cost = JuMP.value.(costs);
        totale = JuMP.value.(totale);
        avg_emissions = sum(totale[t] * data.weights[t]/sum(data.weights) for t=1:T);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "avg_emissions" => avg_emissions,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "totale" => totale,
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

In [ ]:
results_tax = clear_market_invest_tax(dfclust, tech)

Finally, we will add a subsidy to the model. One can think of the subsidy as a negative cost to renewable power. 

In [11]:
function clear_market_invest_subsidy(data::DataFrame, tech::DataFrame,subsidy=10.0)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);

    #New Variable
    @variable(model, totale[1:T]); #total emissions
    
    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                (gross_surplus[t] - costs[t] ) for t=1:T)
             - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw 
             - tech.F2[5]*gas_gw^2  - tech.F2[6]*wind_gw^2 - tech.F2[7]*solar_gw^2);

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * price[t]);
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
                (data.a[t] - demand[t]) * demand[t] / data.b[t] 
            + demand[t]^2/(2*data.b[t]));

    # Define cost
    @constraint(model, [t=1:T], costs[t] ==
                    sum(tech.c[i] * quantity[t,i]
                    + tech.c2[i] * quantity[t,i]^2/2 
                    - subsidy * tech.renewable[i] * quantity[t,i] for i=1:I)
        + (imports[t] - data.am[t])^2/(2 * data.bm[t]))

    # Define emissions 
    @constraint(model, [t=1:T], totale[t] ==
                    sum(tech.e[i] * quantity[t,i] for i=1:I))


        # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);


    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        avg_demand = sum(d[t] * data.weights[t]/sum(data.weights) for t=1:T);
        cost = JuMP.value.(costs);
        totale = JuMP.value.(totale);
        avg_emissions = sum(totale[t] * data.weights[t]/sum(data.weights) for t=1:T);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "avg_emissions" => avg_emissions,
            "avg_demand" => avg_demand,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "totale" => totale,
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

clear_market_invest_subsidy (generic function with 2 methods)

A subsidy decrease wholesale prices and thus increase consumption. However, this consumption negative externality is not priced. Thus, on what follows, we add a budget balance condition in which retail prices have to cover the total payment to electricity producers.

In [12]:
results_subsidy = clear_market_invest_subsidy(dfclust, tech)

Dict{String, Any} with 13 entries:
  "avg_price"     => 32.7805
  "price"         => [39.7889, 29.318, 30.0876, 40.2601, 30.2362, 34.9357, 30.5…
  "gas_gw"        => 0.996083
  "status"        => "LOCALLY_SOLVED"
  "avg_demand"    => 26.6568
  "totale"        => [7.49409, 4.49075, 4.49075, 7.75092, 4.49075, 4.84853, 4.4…
  "quantity"      => [9.18815 11.5 … 1.05553 4.88794e-9; 3.42648 11.5 … 1.71361…
  "solar_gw"      => 0.0
  "imports"       => [8.94036, 8.3391, 7.52827, 6.94479, 4.94853, 8.32427, 8.03…
  "demand"        => [37.1794, 25.9753, 27.1204, 30.115, 22.8582, 25.063, 26.68…
  "avg_emissions" => 5.14646
  "cost"          => [645.482, 364.573, 409.749, 603.99, 339.42, 416.4, 354.061…
  "wind_gw"       => 4.63653

In [13]:
function clear_market_invest_subsidy_bb(data::DataFrame, tech::DataFrame,subsidy=10.0,
    fee = 0.1)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);

    #New Variable
    @variable(model, totale[1:T]); #total emissions
    
    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                (gross_surplus[t] - costs[t] ) for t=1:T)
             - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw 
             - tech.F2[5]*gas_gw^2  - tech.F2[6]*wind_gw^2 - tech.F2[7]*solar_gw^2);

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * (1 + fee) * price[t]);
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * (1 + fee) * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
                (data.a[t] - demand[t]) * demand[t] / data.b[t] 
            + demand[t]^2/(2*data.b[t]));

    # Define cost
    @constraint(model, [t=1:T], costs[t] ==
                    sum(tech.c[i] * quantity[t,i]
                    + tech.c2[i] * quantity[t,i]^2/2 
                    - subsidy * tech.renewable[i] * quantity[t,i] for i=1:I)
        + (imports[t] - data.am[t])^2/(2 * data.bm[t]))

    # Define emissions 
    @constraint(model, [t=1:T], totale[t] ==
                    sum(tech.e[i] * quantity[t,i] for i=1:I))


    # Budget balance
    @constraint(model,[t=1:T],
    fee * demand[t] == sum(subsidy * tech.renewable[i] * quantity[t,i] for i=1:I))
    
    # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);


    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        avg_demand = sum(d[t] * data.weights[t]/sum(data.weights) for t=1:T);
        cost = JuMP.value.(costs);
        totale = JuMP.value.(totale);
        avg_emissions = sum(totale[t] * data.weights[t]/sum(data.weights) for t=1:T);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "avg_emissions" => avg_emissions,
            "avg_demand" => avg_demand,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "totale" => totale,
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

clear_market_invest_subsidy_bb (generic function with 3 methods)

In [14]:
results_subsidy_bb = clear_market_invest_subsidy_bb(dfclust, tech)

Dict{String, Any} with 13 entries:
  "avg_price"     => 30.9542
  "price"         => [35.9063, 30.8191, 24.4301, 36.3112, 31.7768, 31.1772, 31.…
  "gas_gw"        => 1.73032
  "status"        => "LOCALLY_SOLVED"
  "avg_demand"    => 26.5558
  "totale"        => [7.74514, 4.74773, 4.72949, 7.99523, 5.26135, 4.88631, 5.3…
  "quantity"      => [9.18815 11.5 … 0.320084 0.0519353; 3.42648 11.5 … 0.25575…
  "solar_gw"      => 0.158706
  "imports"       => [8.92301, 8.70137, 7.28762, 6.92892, 5.15299, 8.26795, 8.3…
  "demand"        => [37.2019, 25.6143, 27.409, 30.1374, 22.5632, 25.1281, 26.3…
  "avg_emissions" => 5.54145
  "cost"          => [668.669, 408.071, 420.538, 626.617, 417.979, 420.095, 444…
  "wind_gw"       => 3.3139

## Follow-up exercises

1. Consider a tax and a subsidy that reach the same target of emissions. What are the different costs? How are the different components of welfare affected?

Note: This will require you to include emissions as an input or an output to the function.